# OS Features API 

This notebook is using some simple packages and tools to select an area of interest and return the features within it from the OS Features API.

**Note: There is a known issue when returning Linestring geometry types** 

***

First of all we need to import the packages we need to be able to carry out the function in this notebook. Depending on how you set up your environment you will need install the packages below. Suggested method either using pip or conda.

In [1]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer,GeoData, LayersControl, WMSLayer,ImageOverlay
from shapely.geometry.polygon import Polygon
import geopandas as gpd, pandas as pd
import requests as r
from datetime import datetime
from urllib.parse import unquote, urlencode
#from owslib.wfs import WebFeatureService as wfs

This next stage sets out some of the variables that we'll pass to various calls and functions in the rest of the notebook. 

Please Enter your API key below 

In [2]:
apikey = #enter API key

#The type of request we're making to the API, when called use request[0] where the 0 is the list position of the argument you want
request = ['GetFeature','DescribeFeatureType','GetCapabilities']

#Standard OS API Call
api_url = 'https://api.os.uk/features/v1/wfs?service=wfs&version=2.0.0&request='

#The count of feature we want to return, max = 100. 
count = 100

#Spatial function used. Options = Equals, Disjoint, Touches, Within, Overlaps, Crosses, Intersects, and Contains
method = 'Within'

#This is the max number of features you want to call, the API returns 100 at a time, max should be 60,000 but this is untested currently. 
max_features = 1000

#The layer name you wish to return  ##Future development add a drop down of available layers
typeNames = 'Zoomstack_LocalBuildings'#'Highways_RoadNode'#'OpenUPRN_Address'##'Highways_RoadLink'

The following section now adds in some of the mapping variables, here you can choose which WMTS you'd like to use. The example uses Light_3857. We also set the style of the box we're about to draw on the map. 

There is a function defined in this section that records the shape of a bounding box. Once drawn the map will automatically close as its not required until the end again. 



In [3]:
params = urlencode({'key': apikey,
                    'service': 'WMTS',
                    'request': 'GetTile',
                    'version': '2.0.0',
                    'height': 256,
                    'width': 256,
                    'outputFormat': 'image/png',
                    'style': 'default',
                    # Example uses Light Style in Web Mercator (EPSG:3857) projection
                    'layer': 'Light_3857',
                    'tileMatrixSet': 'EPSG:3857',
                    'tileMatrix': '{z}',
                    'tileRow': '{y}',
                    'tileCol': '{x}'})

# OS Data Hub base path - https://api.os.uk
# OS Maps API WMTS end point path - /maps/raster/v1/wmts?
url = unquote(f'https://api.os.uk/maps/raster/v1/wmts?{params}')
# Create custom ipyleaflet TileLayer for the OS Maps API WMTS resource
os_maps_api = {'url': url,
               'min_zoom': 7,
               'max_zoom': 20,
               'attribution': f'Contains OS data &copy; Crown copyright and database rights {datetime.now().year}'}
# OS logo image
image = ImageOverlay(url='https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.svg',
                     bounds=((0.0, 0.0), (20, 10)))
# Create ipyleaflet Map
m = Map(basemap=os_maps_api,
        center=[51.507, -2.105],
        zoom=7)

# Add image to Map
m.add_layer(image)

#Apply the draw control function to a variable for ease
draw_control = DrawControl()

#style of a rectangle for the map
draw_control.rectangle = {
"shapeOptions": {
"fillColor": "#efed69",
"color": "#efed69",
"fillOpacity": 0.5}}

#Add the draaw control to the map 
m.add_control(draw_control)

#Create an empty GeoDataFrame for the funtion to store the bounding box
poly_union = gpd.GeoDataFrame()

#Create handle_draw function which carries out actions after drawing a shape on the map. 
def handle_draw(self, action, geo_json):
    global poly_union
    my_poly=draw_control.last_draw['geometry']['coordinates'][0]
    geom = Polygon(my_poly)
    print(geom)
    poli= gpd.GeoDataFrame(geometry=[geom])
    poly_union=pd.concat([poly_union,poli],sort=False)
    poly_union = poly_union.set_crs("EPSG:4326")
    poly_union = poly_union.to_crs("EPSG:27700")
    m.close()

#on draw on the map call the function
draw_control.on_draw(handle_draw)

#draw the map
m


Map(center=[51.507, -2.105], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

Now we've defined our area of interest we can run the API for the variables dictated in earlier sections

In [6]:
#assign bounds to variables
bx = poly_union.bounds['minx']
by = poly_union.bounds['miny']
tx = poly_union.bounds['maxx']
ty = poly_union.bounds['maxy']

#create a list of startIndex's
startIndex = list(range(0, max_features, 100))

#Create empty dataframe for api results
api_data = gpd.GeoDataFrame()

#Run api loop to return features as disctated by variables
for index in startIndex:
    url = f'{api_url}{request[0]}&typeNames={typeNames}&count={count}&key={apikey}&outputFormat=GeoJSON&startIndex={index}&filter=<ogc:Filter><ogc:{method}><gml:Box%20srsName="EPSG:27700"><gml:coordinates%20decimal="."%20cs=","%20ts=",">{bx[0]},{by[0]},{tx[0]},{ty[0]}</gml:coordinates></gml:Box></ogc:Within></ogc:Filter>'
    data = r.get(url)
    json = data.json()
    gdf = gpd.GeoDataFrame.from_features(json['features'])
    api_data = api_data.append(gdf)
    
#set crs and reproject
api_data = api_data.set_crs("EPSG:27700")
repro = api_data.to_crs("EPSG:4326") 

In this final stage we add the data from the API call to a map and display

We could add in a section here that just copies the dataframe to a database using geopandas.to_postgis (would rely on setting up a SQLAlchemy engine)

In [9]:
geo_data = GeoData(geo_dataframe = repro,
                  name = typeNames)
m2 = Map(basemap=os_maps_api,
    center=[51.507, -2.105],
    zoom=7)

m2.add_layer(geo_data)
m2.add_control(LayersControl())
m2